In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

In [2]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
G

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [5]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReviewAnalysis").getOrCreate()

In [6]:
# There is a DataFrame or table for the vine_table data using one of three methods above
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Row Count
video_games_df.count()

960872

In [7]:
from pyspark.sql.functions import col, avg

# Filter by voters.
filtered_video_games_df = video_games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_video_games_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [8]:
# The data is filtered to create a DataFrame or table where there are 20 or more total votes (5 pt)
greater_than_twenty_df = filtered_video_games_df.filter(filtered_video_games_df['total_votes'] >= 20)
greater_than_twenty_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [9]:
# The data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50% (5 pt)
fifty_percent_cut_df = greater_than_twenty_df.filter(greater_than_twenty_df["helpful_votes"]/greater_than_twenty_df["total_votes"] >= 0.5)
fifty_percent_cut_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

In [10]:
# The data is filtered to create a DataFrame or table where there is a Vine review (5 pt)
vine_review_df = fifty_percent_cut_df.filter(fifty_percent_cut_df['vine']== 'Y')
vine_review_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|
|R1FNVUXPU63WOZ|          4|           43|         48|   Y|                N|
|R25XGG2G12SE1Z|          4|           20|         23|   Y|                N|
| R3JKU4HRDFZDH|          4|           27|         30|   Y|                N|
|R2PQYOCJXRB1BF|          5|           26|         28|   Y|     

In [11]:
# The data is filtered to create a DataFrame or table where there isn’t a Vine review (5 pt)
no_vine_review_df = fifty_percent_cut_df.filter(fifty_percent_cut_df['vine']== 'N')
no_vine_review_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|     

In [12]:
# The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews (15 pt)

# Determine the percentage of five-star reviews among Vine reviews
vine_number = vine_review_df.count()
vine_five_star_number = vine_review_df.filter(vine_review_df['star_rating']== 5).count()
percentage_five_star_vine = float(vine_five_star_number) / float(vine_number)
print(vine_number)
print(vine_five_star_number)
print(percentage_five_star_vine)

# Determine the percentage of five-star reviews among non-Vine reviews.
no_vine_number = no_vine_review_df.count()
no_vine_five_star_number = no_vine_review_df.filter(no_vine_review_df['star_rating']== 5).count()
percentage_five_star_no_vine = float(no_vine_five_star_number) / float(no_vine_number)
print(no_vine_number)
print(no_vine_five_star_number)
print(percentage_five_star_no_vine)

# Total amount of reviews
print(vine_number + no_vine_number)

47
15
0.3191489361702128
8362
4332
0.5180578808897393
8409
